In [ ]:
# 最终正确版 - 每篇publication作为独立行
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import nest_asyncio
import random

nest_asyncio.apply()

async def human_delay(min_sec=2, max_sec=4):
    """Random delay"""
    await asyncio.sleep(random.uniform(min_sec, max_sec))

async def scrape_professor_complete(page, prof_url, prof_name):
    """Visit professor page and extract all information, each publication as separate return"""
    print(f"  Visiting: {prof_name}")
    
    # Basic information
    basic_info = {
        'name': prof_name,
        'profile_url': prof_url,
        'title': 'N/A',
        'department': 'N/A'
    }
    
    publications = []  # Store all publications
    
    try:
        # Visit personal page
        try:
            await page.goto(prof_url, wait_until='load', timeout=120000)
        except:
            print(f"    Load timeout, trying to continue...")
        
        await asyncio.sleep(3)
        
        # Scroll to load content
        for _ in range(3):
            try:
                await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
                await asyncio.sleep(1)
            except:
                pass
        
        # ===== Extract Title =====
        try:
            title_elem = await page.query_selector('.field--name-field-faculty-title .field__item')
            if title_elem:
                basic_info['title'] = (await title_elem.inner_text()).strip()
        except:
            pass
        
        # ===== Extract Department =====
        try:
            # Method 1: Find via field--name-field-academic-area-full
            dept_field = await page.query_selector('.field--name-field-academic-area-full .field__item')
            if dept_field:
                basic_info['department'] = (await dept_field.inner_text()).strip()
            else:
                # Method 2: Find "Academic Areas" label
                dept_label = await page.query_selector('text="Academic Areas"')
                if dept_label:
                    parent = await dept_label.evaluate_handle('el => el.closest(".field")')
                    if parent:
                        dept_item = await parent.query_selector('.field__item')
                        if dept_item:
                            basic_info['department'] = (await dept_item.inner_text()).strip()
        except Exception as e:
            print(f"    Failed to extract Department: {str(e)}")
        
        # ===== Extract Publications - Only from Articles tab =====
        try:
            # Method 1: Find directly from div with id="tabContentPaper"
            articles_tab_content = await page.query_selector('#tabContentPaper')
            
            pub_articles = []
            if articles_tab_content:
                # Find article tags only within this div
                pub_articles = await articles_tab_content.query_selector_all('article.publication.publication--default')
                print(f"    Found {len(pub_articles)} articles in Articles tab")
            else:
                # Backup: Click Articles tab then search
                try:
                    articles_button = await page.query_selector('#tabNavPaper')
                    if articles_button:
                        await articles_button.click()
                        await asyncio.sleep(1)
                        articles_tab_content = await page.query_selector('#tabContentPaper')
                        if articles_tab_content:
                            pub_articles = await articles_tab_content.query_selector_all('article.publication.publication--default')
                            print(f"    Found {len(pub_articles)} articles after clicking")
                except:
                    print(f"    Unable to locate Articles tab")
            
            for idx, article in enumerate(pub_articles, 1):
                try:
                    pub_data = {}
                    
                    # Extract title
                    title_elem = await article.query_selector('.publication__title')
                    if title_elem:
                        title_text = await title_elem.inner_text()
                        pub_data['pub_title'] = title_text.strip()
                    else:
                        continue  # Skip if no title
                    
                    # Extract Published Date
                    date_elem = await article.query_selector('.field--name-field-display-date .field__item')
                    if date_elem:
                        pub_data['published_date'] = (await date_elem.inner_text()).strip()
                    else:
                        pub_data['published_date'] = ''
                    
                    # Extract Authors
                    authors_elem = await article.query_selector('.field--name-field-author-text .field__item')
                    if authors_elem:
                        pub_data['authors'] = (await authors_elem.inner_text()).strip()
                    else:
                        pub_data['authors'] = ''
                    
                    # Extract Source
                    journal_elem = await article.query_selector('.field--name-field-journal-title')
                    if journal_elem:
                        pub_data['source'] = (await journal_elem.inner_text()).strip()
                    else:
                        pub_data['source'] = ''
                    
                    # Extract Volume
                    volume_elem = await article.query_selector('.field--name-field-volume .field__item')
                    if volume_elem:
                        pub_data['volume'] = (await volume_elem.inner_text()).strip()
                    else:
                        pub_data['volume'] = ''
                    
                    # Extract Issue
                    issue_elem = await article.query_selector('.field--name-field-issue .field__item')
                    if issue_elem:
                        pub_data['issue'] = (await issue_elem.inner_text()).strip()
                    else:
                        pub_data['issue'] = ''
                    
                    # Extract Pages
                    pages_elem = await article.query_selector('.field--name-field-pages .field__item')
                    if pages_elem:
                        pub_data['pages'] = (await pages_elem.inner_text()).strip()
                    else:
                        pub_data['pages'] = ''
                    
                    publications.append(pub_data)
                    
                except Exception as e:
                    print(f"    Failed to extract publication #{idx}: {str(e)}")
                    continue
        
        except Exception as e:
            print(f"    Failed to extract Publications: {str(e)}")
        
        print(f"    Title: {basic_info['title']}")
        print(f"    Department: {basic_info['department']}")
        print(f"    Publications: {len(publications)} articles")
        
        return basic_info, publications
        
    except Exception as e:
        print(f"    Error: {str(e)}")
        return basic_info, []

async def scrape_all_faculty(csv_file='umich_ross_final.csv', start_from=0, end_at=None):
    """
    Scrape complete information for all professors, each publication as a separate row
    
    DataFrame format:
    | name | profile_url | title | department | pub_title | published_date | authors | source | volume | issue | pages |
    """
    
    # Read existing data
    try:
        df_basic = pd.read_csv(csv_file)
        print(f"Read {len(df_basic)} professors' basic information\n")
    except FileNotFoundError:
        print(f"File not found: {csv_file}")
        return None
    
    if end_at is None or end_at > len(df_basic):
        end_at = len(df_basic)
    
    print(f"Will process professors {start_from+1} to {end_at}")
    print(f"{'='*70}\n")
    
    all_rows = []  # Store all rows (one row per publication)
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        
        context = await browser.new_context(
            viewport={'width': 1920, 'height': 1080},
            user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        )
        
        await context.add_init_script("""
            Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
        """)
        
        page = await context.new_page()
        
        try:
            for idx in range(start_from, end_at):
                row = df_basic.iloc[idx]
                name = row['name']
                url = row['profile_url']
                
                print(f"[{idx+1}/{len(df_basic)}] {name}")
                
                # Get detailed information
                basic_info, publications = await scrape_professor_complete(page, url, name)
                
                # If has publications, create one row per publication
                if publications:
                    for pub in publications:
                        combined_row = {
                            'name': basic_info['name'],
                            'profile_url': basic_info['profile_url'],
                            'title': basic_info['title'],
                            'department': basic_info['department'],
                            'pub_title': pub.get('pub_title', ''),
                            'published_date': pub.get('published_date', ''),
                            'authors': pub.get('authors', ''),
                            'source': pub.get('source', ''),
                            'volume': pub.get('volume', ''),
                            'issue': pub.get('issue', ''),
                            'pages': pub.get('pages', '')
                        }
                        all_rows.append(combined_row)
                else:
                    # If no publications, create one row with empty publication fields
                    combined_row = {
                        'name': basic_info['name'],
                        'profile_url': basic_info['profile_url'],
                        'title': basic_info['title'],
                        'department': basic_info['department'],
                        'pub_title': 'No publications found',
                        'published_date': '',
                        'authors': '',
                        'source': '',
                        'volume': '',
                        'issue': '',
                        'pages': ''
                    }
                    all_rows.append(combined_row)
                
                # Save progress every 10 professors
                if (idx + 1) % 10 == 0:
                    df_temp = pd.DataFrame(all_rows)
                    df_temp.to_csv('umich_ross_detailed_temp.csv', index=False, encoding='utf-8-sig')
                    print(f"\nSaved progress: {len(all_rows)} rows\n")
                
                # Delay
                await human_delay(3, 6)
            
            # Create final DataFrame
            df_final = pd.DataFrame(all_rows)
            
            # Save
            df_final.to_csv('umich_ross_detailed_final.csv', index=False, encoding='utf-8-sig')
            try:
                df_final.to_excel('umich_ross_detailed_final.xlsx', index=False, engine='openpyxl')
            except:
                print("Unable to save Excel file")
            
            print(f"\n{'='*70}")
            print(f"Complete!")
            print(f"Total professors: {len(df_basic[start_from:end_at])}")
            print(f"Total rows: {len(all_rows)} (one row per publication)")
            print(f"Data saved to:")
            print(f"   - umich_ross_detailed_final.csv")
            print(f"   - umich_ross_detailed_final.xlsx")
            print(f"{'='*70}\n")
            
            return df_final
            
        except KeyboardInterrupt:
            print("\nUser interrupted, saving current progress...")
            if all_rows:
                df = pd.DataFrame(all_rows)
                df.to_csv('umich_ross_detailed_interrupted.csv', index=False, encoding='utf-8-sig')
            return pd.DataFrame(all_rows) if all_rows else None
            
        except Exception as e:
            print(f"\nError: {str(e)}")
            if all_rows:
                df = pd.DataFrame(all_rows)
                df.to_csv('umich_ross_detailed_error.csv', index=False, encoding='utf-8-sig')
            return pd.DataFrame(all_rows) if all_rows else None
            
        finally:
            await browser.close()

# =============================================================================
# Complete Usage Guide
# =============================================================================

# [Step 1] Ensure you have CSV file with all professors' names and URLs
# If not, run this first (in another cell):
# df_basic = await run_scraper(total_pages=9, fetch_publications=False)

# [Step 2] Test with first 3 professors to ensure everything works
# df_test = await scrape_all_faculty('umich_ross_final.csv', start_from=0, end_at=3)
# display(df_test)

# [Step 3] Scrape all! Expected 1-2 hours, auto-saves progress
df_all = await scrape_all_faculty('umich_ross_final.csv')

# [Step 4] View final results
if df_all is not None:
    print("\n" + "="*70)
    print("Success!")
    print("="*70)
    print(f"Total rows: {len(df_all)}")
    print(f"Total professors: {df_all['name'].nunique()}")
    print(f"Total publications: {len(df_all[df_all['pub_title'] != 'No publications found'])}")
    print("\nData preview:")
    display(df_all.head(20))
    
    # Statistics by professor
    pub_counts = df_all[df_all['pub_title'] != 'No publications found'].groupby('name').size()
    print(f"\nPublications statistics:")
    print(f"  Average per professor: {pub_counts.mean():.1f} articles")
    print(f"  Maximum: {pub_counts.max()} articles")
    print(f"  Minimum: {pub_counts.min()} articles")

# =============================================================================
# Advanced Usage
# =============================================================================

# If interrupted, continue from position 50:
# df_resume = await scrape_all_faculty('umich_ross_final.csv', start_from=50)

# Scrape specific range (e.g., positions 20-30):
# df_range = await scrape_all_faculty('umich_ross_final.csv', start_from=20, end_at=30)

# View all publications for a specific professor:
# prof_name = "Allan Afuah"  # Replace with desired professor
# prof_pubs = df_all[df_all['name'] == prof_name]
# display(prof_pubs)

# Export data for specific department:
# strategy_profs = df_all[df_all['department'] == 'Strategy']
# strategy_profs.to_csv('strategy_faculty.csv', index=False, encoding='utf-8-sig')

In [3]:
import pandas as pd

# 读取你的解析后的 Excel
df = pd.read_excel("umich_ross_detailed_final.xlsx")

# 打印前 10 行
print(df.head(10))


           name                                        profile_url  \
0   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
1   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
2   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
3   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
4   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
5   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
6   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
7   Allan Afuah  https://michiganross.umich.edu/faculty-researc...   
8  Hyun-Soo Ahn  https://michiganross.umich.edu/faculty-researc...   
9  Hyun-Soo Ahn  https://michiganross.umich.edu/faculty-researc...   

                                    title                 department  \
0                   Professor of Strategy                   Strategy   
1                   Professor of Strategy                   Strategy   
2            